# Uncertainty Analysis over Viscosity Calculation
The following cells serve for performing simulations with which the uncertainty coming from the different elements of the circuit,as well as the measurements from the sensors and other parasite effects, is analyzed.

The effect of these defects is studied for the step response situation. When a step signal is inserted into the system, it is assumed that this will have an exponential response until it stabilizes in some steady state values. The defects on the elements and other parasite effects (resistances or capacities not included in the analytical model) will have an impact over the real response of the system, affecting the final viscosity estimation.

The content of this notebook is the following:
- Assignation of values for all the elements of the circuits and their uncertainty ranges.
- Definition of all the equations involved in the analysis.
- Definition of the simulation rutine.
- Monte Carlo analysis routine.
- Results analysis.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

## 1. Elements of the circuit
The simulated circuit is showing here:

*Insert a diagram of the circuit*

Each dictionary holds the required values to define the associated element of the circuit. Within the dictionary there is a list called `"e_sources"`, or error sources. This list will contain all the parameters to be randomly modified in the Monte Carlo analysis. To do so, the parameter must have an associated parameter `_e`, which defined the absolute error associated to the parameter.

For example, if the diameter, `D`, of the first resistance, `R0`, must be analyzed, it must have an associated parameter `D_e`, defining the absolute uncertainty of that parameter. Also, `D` must be then included in the list `e_sources` of `R0`.

**Note**: It is important to respect the nomenclature in the dictionary of parameters. If it is a resistive element, it must start by `R`. If it is capacitive, `C`. After the identificative number, use a low bar, `_`, for the second part of the name (e.g. `R0_params`).

In [2]:
temp_C = 25.65 # In Celsius
temp_K = temp_C + 273.15

# Input resistance
elements = {
    "R0_params": {
        "D": 500e-6,
        "L": 25e-2,
        "e_sources": [],
        "e": {
            "D_e": 50e-6,
            "L_e": 1e-3
        }
    },

    # First RC Branch
    "R1_params": {
        "D": 150e-6,
        "L": 10e-2,
        "e_sources": [],
        "e": {
            "D_e": 15e-6,
            "L_e": 1e-3
        }
    },
    "C1_params": {
        "D": 2e-3,
        "L": 2.5e-2,
        "E": 10e6,
        "t": 0.75e-3,
        "beta": 2.5e9,
        "e_sources": ["L"],
        "e": {
            "D_e": 0.0,
            "L_e": 1.5e-2
        }
    },

    # Second RC branch
    "R2_params": {
        "D": 150e-6,
        "L": 10e-2,
        "e_sources": [],
        "e": {
            "D_e": 15e-6,
            "L_e": 1e-3
        }
    },
    "C2_params": {
        "D": 2e-3,
        "L": 10e-2,
        "E": 10e6,
        "t": 0.75e-3,
        "beta": 2.5e9,
        "e_sources": [],
        "e": {
            "D_e": 5e-4,
            "L_e": 1e-3
        }
    },
}

In [3]:
# Use the above dictionaries to define the ranges
ranges_dict = {}
for ky, element in elements.items():
    ranges_dict[ky] = {}
    for param in set(element.keys())-set(["e_sources", "e"]):
        if param in element["e_sources"]:
            ranges_dict[ky][param] = (element[param] - element["e"][param+"_e"],
                                      element[param] + element["e"][param+"_e"])
        else: 
            ranges_dict[ky][param] = (element[param], element[param])

print(ranges_dict)

{'R0_params': {'D': (0.0005, 0.0005), 'L': (0.25, 0.25)}, 'R1_params': {'D': (0.00015, 0.00015), 'L': (0.1, 0.1)}, 'C1_params': {'D': (0.002, 0.002), 'beta': (2500000000.0, 2500000000.0), 'E': (10000000.0, 10000000.0), 'L': (0.010000000000000002, 0.04), 't': (0.00075, 0.00075)}, 'R2_params': {'D': (0.00015, 0.00015), 'L': (0.1, 0.1)}, 'C2_params': {'D': (0.002, 0.002), 'beta': (2500000000.0, 2500000000.0), 'E': (10000000.0, 10000000.0), 'L': (0.1, 0.1), 't': (0.00075, 0.00075)}}


## 2. Equations

In [4]:
class calc_water_viscosity:
    """
    Calculates the viscosity at some temperature, in Kelvin.
    """
    def __init__(self, A=2.414e-5, B=247.8, C=140):
        self.A = A
        self.B = B
        self.C = C
    def __call__(self, temp_K):
        return self.A * 10 ** (self.B / (temp_K - self.C))

def calc_cil_resistance(D, L, visc):
    """
    Calculates the hydraulic resistance for a cylindrical tube,
    for a given viscosity.
    """
    return 8*visc*L / (np.pi*((D/2)**4))

def calc_cil_capacitance(D, L, beta, E, t):
    """
    Calculates the theoretical capacitance of a cylindrical tube.
    """
    # Beta := Bulk Modulus of Water
    V0 = np.pi*(D/2)**2 * L
    return (V0 / beta)*( 1 + ((beta*D)/(E*t)) )

def circuit_dyn_eqs(t, dP, args): # R0, R1, R2, C1, C2, Pin
    """
    Calcualtes the differential values of pressure drop over the
    two RC branches of the circuit.
    """
    P1, P2 = dP # Integration of previous dP1, dP2
    R0 = args["R0"]
    R1 = args["R1"]
    R2 = args["R2"]
    C1 = args["C1"]
    C2 = args["C2"] 
    Pin = args["Pin"]

    dP1 = (Pin - P1)/(R0 * C1) - P1 / (R1 * C1)
    dP2 = P1/(R2 * C2) - P2/(R2 * C2)

    return [dP1, dP2]

def estimate_visc(t, P2_t, R2, C2, visc):
    """
    Estimates the viscosity from an step response of the target 
    microfluidic RC circuit. It assumes that there is not noise.

    The viscosity is asked as argument because R2 is assumed to 
    contain the viscosity. 
    """
    P_ss = P2_t[-1]
    if np.abs(P2_t[-1]-P2_t[-2]) > np.abs(0.02*P2_t[-1]):
        print("W: steady state value was not steady calculating visc.")
    P_tau = 0.632 * P_ss
    # Get closest time, t
    tau = t[ np.argmin(np.abs(np.subtract(P2_t, P_tau))) ]

    Rg = R2 / visc
    return tau / (Rg*C2)


## 3. Monte Carlo Analysis
For each of the element of the circuit, there are some defined ranges of values that the element can take. The objective of this analysis is to study how much will affect each of the variations to the output estimation of viscosity.

The analysis is performed by running a large number of simulations (proportional to the complexity of the system and the number of uncertain variables), propagating each variation to an objective value. For example, if the uncertainty source is one dimension of a resistance, the resistance will be calculated for each simulation with a dimension randomly chosen within the defined range. For each simulation, the viscosity will be calculated. The variation at viscosity is then compared with the variation of the input parameter, producing a sensibility estimation between both of them.

In this case, the objective parameter is the parasite capacitance, although the rest of the parameters can be analyzed with this same setup.

In [5]:
Pin = 20e5 # Pressure input

t_sim = 5 # Must be large enough to reach steady state
t_points = 500 # This affects to the accuracy estimating tau
N = 1000 # Number of simulations


visc_calc = calc_water_viscosity()
visc = visc_calc(temp_K)

results = {}
for ky in ranges_dict.keys():
    results[ky.split("_")[0]] = []
results["est_visc"] = []

for n in tqdm(range(N)): # For progress bar
    # Get the values of the parameters
    sim_params = { "Pin": Pin }
    for ky, element_ranges in ranges_dict.items():
        # Extract the random values for the parameters
        args = {}
        for param, param_ranges in element_ranges.items():
            args[param] = np.max([0.0, np.random.uniform(*param_ranges)])
        # Resistance
        if ky[0] == "R":
            sim_params[ky.split("_")[0]] = calc_cil_resistance(**args, visc=visc)
        # Capacitance
        elif ky[0] == "C":
            sim_params[ky.split("_")[0]] = calc_cil_capacitance(**args)
        else: 
            print("Error: element {} not recognized".format(ky))
            exit()
        
    # Perform the simulation
    sol = solve_ivp(circuit_dyn_eqs, [0, t_sim], [0, 0], args=(sim_params,), 
                    dense_output=True)
    
    # Save the results
    t = np.linspace(0, t_sim, t_points)
    P2_t = sol.sol(t)[1]

    # If you want to print some curves, don't make N very large
    # P1_t = sol.sol(t)[0]
    # plt.plot(t, P2_t)
    # plt.plot(t, P1_t)
    # plt.show()

    # Calculate viscosity from P2_t
    est_visc = estimate_visc(t, P2_t, sim_params["R2"], sim_params["C2"], visc)

    # Update the results dict
    for ky in ranges_dict.keys():
        results[ky.split("_")[0]].append( sim_params[ky.split("_")[0]] )
    results["est_visc"].append(est_visc)

res_df = pd.DataFrame.from_dict(results)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [7]:
print("Real viscosity: {}".format(visc))

# print(res_df)

print("Mean values:")
print(res_df.mean())

print("Standard Deviation values:")
print(res_df.std())

Real viscosity: 0.0008773859312223907
Mean values:
R0          1.429917e+11
R1          7.061317e+12
C1          2.125101e-14
R2          7.061317e+12
C2          8.390147e-14
est_visc    8.890848e-04
dtype: float64
Standard Deviation values:
R0          3.052063e-05
R1          0.000000e+00
C1          7.256912e-15
R2          0.000000e+00
C2          0.000000e+00
est_visc    7.390379e-06
dtype: float64
